In [60]:
import forallpeople as si
si.environment('Structural',top_level=True)
#Useful extra units
cm = 1e-2*m #type: ignore
from IPython.display import Markdown, display
import handcalcs.render

In [118]:
b=20*cm
h=50*cm
M_u=50*kN*m
ØM_n=55*kN*m
FU_b=M_u/ØM_n
rebar_b="3Ø12"
A_s=1.13*cm**2*3
display(Markdown(f"Dimensiones $b$={round(b,0)}, $h$={round(h,0)}"))
display(Markdown(f"Demanda $M_u$={round(M_u,0)}, armadura {rebar_b}, $A_s$={round(A_s.value*10000,2)} cm²>$A_{{s.min}} \\color{{green}}{{\\, ✓}}$"))
display(Markdown(f"Capacidad $\phi M_n$={round(ØM_n,0)}, $\\color{{green}}{{\\text{{FU}}={round(FU_b,2)}}} \\, ✓$"))
V_u=40*kN
rebar_v="1eØ6/20"
A_v=2.83*cm**2/m
ØV_n=80*kN
FU_v=V_u/ØV_n
display(Markdown(f"Demanda $V_u$={round(V_u,0)}, armadura {rebar_v}, $A_v$={round(A_v.value*10000,2)} cm²/m>$A_{{v.min}} \\color{{green}}{{\\, ✓}}$"))
display(Markdown(f"Capacidad $\phi V_n$={round(ØV_n,0)}, $\\color{{green}}{{\\text{{FU}}={round(FU_v,2)}}} \\, ✓$"))

Dimensiones $b$=200 mm, $h$=500 mm

Demanda $M_u$=50 kN·m, armadura 3Ø12, $A_s$=3.39 cm²>$A_{s.min} \color{green}{\, ✓}$

Capacidad $\phi M_n$=55 kN·m, $\color{green}{\text{FU}=0.91} \, ✓$

Demanda $V_u$=40 kN, armadura 1eØ6/20, $A_v$=2.83 cm²/m>$A_{v.min} \color{green}{\, ✓}$

Capacidad $\phi V_n$=80 kN, $\color{green}{\text{FU}=0.5} \, ✓$

0.9090909090909091

In [141]:
import math
class FUFormatter:
    def __init__(self, FU):
        self.FU = FU
    
    def format_FU(self):
        # Determine color based on FU value
        if self.FU < 0.95:
            color = '#439b00'
        elif 0.95 <= self.FU <= 1:
            color = '#efc200'
        else:
            color = '#d43e36'
        
        return f"$\\color{{{color}}}{{\\text{{FU}}={round(self.FU,2)}}}$"

class BeamDesign:
    def __init__(self, b, h, M_u):
        self.b = b
        self.h = h
        self.M_u = M_u
    
    @property
    def display_results(self):
        A_s = 1.13*cm**2*3
        rebar_b = "3Ø12"  # Replace with actual rebar type if needed
        ØM_n = 55*kN*m  # Example capacity calculation
        FU_b = self.M_u/ØM_n  # Example Utilization Ratio (FU)
        # Create FUFormatter instance and format FU value
        FU_formatter = FUFormatter(FU_b)
        formatted_FU = FU_formatter.format_FU()
        
        # Print results
        display(Markdown(f"Demanda $M_u$={round(self.M_u,0)}, armadura {rebar_b}, $A_s$={round(A_s.value*10000,2)} cm²>$A_{{s.min}}$"))
        display(Markdown(f"Capacidad $\\phi M_n$={round(ØM_n,0)}, {formatted_FU}")) 

# Example usage
b=20*cm
h=50*cm
M_u=40*kN*m
beam = BeamDesign(b, h, M_u)
beam.display_results
M_u=54*kN*m
beam = BeamDesign(b, h, M_u)
beam.display_results
M_u=70*kN*m
beam = BeamDesign(b, h, M_u)
beam.display_results
A_s_min=2.5*cm**2/m


Demanda $M_u$=40 kN·m, armadura 3Ø12, $A_s$=3.39 cm²>$A_{s.min}$

Capacidad $\phi M_n$=55 kN·m, $\color{#439b00}{\text{FU}=0.73}$

Demanda $M_u$=54 kN·m, armadura 3Ø12, $A_s$=3.39 cm²>$A_{s.min}$

Capacidad $\phi M_n$=55 kN·m, $\color{#efc200}{\text{FU}=0.98}$

Demanda $M_u$=70 kN·m, armadura 3Ø12, $A_s$=3.39 cm²>$A_{s.min}$

Capacidad $\phi M_n$=55 kN·m, $\color{#d43e36}{\text{FU}=1.27}$

In [15]:
import sys
import os
# Añadir el directorio raíz al PYTHONPATH
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../')))

from structurelab.settings import Settings
from structurelab.material import create_concrete, SteelBar
from structurelab.section import Beam

from pylatex import Command, Document, Math, Matrix, Section, Subsection, VectorName, NoEscape, Figure, LongTable
from pylatex.utils import NoEscape
import matplotlib
matplotlib.use("Agg")  # Not to use X server. For TravisCI.
import matplotlib.pyplot as plt  # noq

class MyDocument(Document):
    def __init__(self, documentclass='article', document_options=['a4paper','spanish','10pt'], geometry_options={"left": "20mm","right": "20mm"}):
        # Pasar las opciones al constructor de la clase base Document
        super().__init__(documentclass=documentclass, document_options=document_options, geometry_options=geometry_options)
        self.preamble.append(Command("usepackage","mathtools"))
        self.preamble.append(Command("title", "Viga rectangular a flexión - ACI 318-19"))
        self.preamble.append(Command("author", "StructureLab"))
        self.preamble.append(Command("date", NoEscape(r"\today")))
        self.append(NoEscape(r"\maketitle"))




def main():
    # Ejemplo de uso
    concrete=create_concrete(name="H30",f_c=30*MPa, design_code="ACI 318-19") # type: ignore
    steelBar=SteelBar(name="ADN 420", f_y=420*MPa) # type: ignore
    section = Beam(
        name="V-40x50",
        concrete=concrete,
        steelBar=steelBar,
        width=400 * mm,  # type: ignore
        depth=500 * mm,  # type: ignore
    )
    print(f"Nombre de la sección: {section.get_name()}")

    doc = MyDocument()
    doc.append(NoEscape(
        r"""
        La resistencia nominal a flexión, $M_n$, de una viga de hormigón armado se calcula mediante la siguiente expresión:

        \begin{equation}
        M_n = A_s \cdot f_y \cdot \left( d - \frac{a}{2} \right)
        \end{equation}

        Donde:

        \begin{itemize}
            \item $A_s$ es el área de la armadura de refuerzo (en $\text{mm}^2$).
            \item $f_y$ es el esfuerzo de fluencia del acero de refuerzo (en MPa).
            \item $d$ es la distancia desde la fibra extrema en compresión hasta el centroide del acero de refuerzo (en mm).
            \item $a$ es la profundidad del bloque de compresión equivalente (en mm), que se determina como:
            \begin{equation}
            a = \frac{\beta_1 \cdot c}{2}
            \end{equation}
        \end{itemize}

        La profundidad $c$ se relaciona con la deformación en el acero de refuerzo, y $\beta_1$ es un factor que depende de la resistencia especificada del concreto, $f'_c$, según las disposiciones del ACI 318-19.

        \section{Factor de Resistencia}

        Para asegurar un diseño seguro, se introduce un factor de resistencia, $\phi$, que se aplica a la resistencia nominal:

        \begin{equation}
        M_u \leq \phi \cdot M_n
        \end{equation}

        Donde $M_u$ es el momento flector último requerido debido a las cargas aplicadas, y $\phi$ es el factor de reducción de resistencia, cuyo valor depende del tipo de fallo esperado y puede variar entre 0.65 y 0.90.

        """))
    doc.generate_pdf("Pylatex_test",clean_tex=False)

if __name__ == "__main__":
    main()

Nombre de la sección: V-40x50
